In [25]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
# embed static images in the ipynb
%matplotlib inline 
import os
import sys

# import PyTorch
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
import pytorch_lightning as pl 
# ! pip install pytorch-lightning --quiet


In [52]:
class LitMNIST(pl.LightningModule):
    # ----------------------------------
    # Network Architecture
    # ----------------------------------
    def __init__(self, data_dir: str, lr: float = 1e-2):
        super().__init__()
        self.data_dir = data_dir
        self.lr = lr
        
        # Hard-coded constants
        self.BATCH_SIZE = 65
        self.n_classes = 10
        self.dims = (1, 28, 28)
        self.loss_fn = F.nll_loss
        channels, width, height = self.dims
        
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(channels * width * height, 64),
                nn.Dropout(p = 0.2),
                nn.LeakyReLU(),
            nn.Linear(64, 32),
                nn.LeakyReLU(),
                nn.Dropout(p = 0.4),
            nn.Linear(32, self.n_classes)
        )

    def forward(self, x):
        x = self.fc_layers(x)
        return F.log_softmax(input = x, dim = 1)

    # ----------------------------------
    # Training, validation, and test steps
    # ----------------------------------

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x) 
        loss = self.loss_fn(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        # compute loss
        logits = self(x)
        loss = self.loss_fn(logits, y)
        # compute accuracy
        y_hat = self.predict(x)
        accuracy = self.accuracy(y_hat, y)
        # self.log interacts with TensorBoard
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', accuracy, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = self.lr)
        return optimizer

    def predict(self, x):
        logits = self(x)
        preds = torch.argmax(logits, dim=1)
        return preds

    def accuracy(self, y_hat, y):
        return pl.metrics.functional.accuracy(y_hat, y)
        
    # ----------------------------------
    # Data preparation hooks
    # ----------------------------------
    def train_dataloader(self) -> torch.utils.data.DataLoader:
        dl = torch.utils.data.DataLoader(
            self.train_set, self.BATCH_SIZE)
        return dl
    def val_dataloader(self) -> torch.utils.data.DataLoader:
        dl = torch.utils.data.DataLoader(
            self.val_set, self.BATCH_SIZE)
        return dl
    def test_dataloader(self) -> torch.utils.data.DataLoader:
        dl = torch.utils.data.DataLoader(
            self.test_set, self.BATCH_SIZE)
        return dl
    
    def setup(self, stage=None):
        if stage in ['fit', None]:
            train_data = torchvision.datasets.MNIST(
                os.getcwd(), train=True, download=True, 
                transform = torchvision.transforms.ToTensor()
            )
            splits = np.array([15, 70]) / 85 * len(train_data)
            splits = splits.astype(int)
            splits[-1] += len(train_data) - splits.sum() 
            train_val_set = torch.utils.data.random_split(
                dataset = train_data,
                lengths = splits)
            self.train_set, self.val_set = train_val_set
        if stage in ['test', None]:
            test_set = torch.vision.datasets(
                os.getcwd(), train=False, download=False,
                transform = torchvision.transforms.ToTensor()
            )
            self.test_set = test_set
        


In [57]:

model = LitMNIST(data_dir = os.path.join(os.getcwd(), "temp"))
trainer = pl.Trainer(gpus=0, max_epochs=3, 
                     progress_bar_refresh_rate=0)
trainer.fit(model)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name      | Type       | Params
-----------------------------------------
0 | fc_layers | Sequential | 52.6 K
-----------------------------------------
52.6 K    Trainable params
0         Non-trainable params
52.6 K    Total params


1

Why is it called logits in `training_step`?
> "Logits are interpreted to be the unnormalised (or not-yet normalised) predictions (or outputs) of a model. These can give results, but we don't normally stop with logits, because interpreting their raw values is not easy...  
The layers of a neural network commonly take input data, multiply that by some parameters (weights) that we want to learn, then apply a non-linearity function, which provides the model with the power to learn non-linear relationships." - [source]

[source]: https://datascience.stackexchange.com/questions/31041/what-does-logits-in-machine-learning-mean

'c:\\Users\\uniqu\\Adaptation\\github repos\\Bioinformatics-Neural Networks for Genomic Risk\\temp'

In [29]:
os.getcwd()

str

In [31]:
type(os.path.join(os.getcwd(), "data"))

str